In [1]:
import tensorflow as tf
print(tf.config.list_physical_devices())
import dgl
from gnn.models.tools import load_data
import numpy as np

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:XLA_CPU:0', device_type='XLA_CPU'), PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:2', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:3', device_type='GPU'), PhysicalDevice(name='/physical_device:XLA_GPU:0', device_type='XLA_GPU'), PhysicalDevice(name='/physical_device:XLA_GPU:1', device_type='XLA_GPU'), PhysicalDevice(name='/physical_device:XLA_GPU:2', device_type='XLA_GPU'), PhysicalDevice(name='/physical_device:XLA_GPU:3', device_type='XLA_GPU')]


Using backend: tensorflow


In [ ]:
g, features, labels, train_mask, val_mask, test_mask = load_data('cora')

print('\nGraph:\n{}'.format(g))
print('\nFeatures:\n{}'.format(features))
print('\nLabels:\n{}'.format(labels))
print('\nTrain mask:\n{}'.format(train_mask))
print('\nVal mask:\n{}'.format(val_mask))
print('\nTest mask:\n{}'.format(test_mask))

In [ ]:
node_ids = g.nodes()
print('Node ids:\n{}'.format(node_ids))

train_nids = []
index = 0
for m in train_mask.numpy():
    if m:
        train_nids.append(node_ids[index])
    else:
        pass
    index += 1

In [ ]:
import copy

batch_size = 32
num_neighbors = 4
num_gcn_layers = 3



# g, features, labels, train_mask, val_mask, test_mask = load_data('cora')
# legacy_graph = dgl.DGLGraphStale(g, readonly=True)


# for nf in dgl.contrib.sampling.NeighborSampler(legacy_graph, 
#                                                batch_size=batch_size,
#                                                expand_factor=num_neighbors,
#                                                neighbor_type='in',
#                                                shuffle=True,
#                                                num_hops=num_gcn_layers,
#                                                seed_nodes=train_nids):
#     pass

batch_size = 20
minibatches_of_train_nids = []
prev_index = 0
index = batch_size
for batch_num in range(int(len(train_nids)/batch_size)):
    minibatches_of_train_nids.append(train_nids[prev_index:index])
    prev_index = copy.deepcopy(index)
    index += batch_size
    
    


sampler = dgl.dataloading.MultiLayerFullNeighborSampler(num_gcn_layers)
collator = dgl.dataloading.NodeCollator(g, train_nids, sampler)
for batch_of_nodes in minibatches_of_train_nids:
    print('\n\nbatch of nodes:\n{}'.format(batch_of_nodes))
    input_nodes, output_nodes, blocks = collator.collate(batch_of_nodes)
    print('\ninput nodes:\n{}'.format(input_nodes))
    print('\noutput nodes:\n{}'.format(output_nodes))
    print('\nblocks:\n{}'.format(blocks))


# dataloader = dgl.dataloading.NodeDataLoader(g, 
#                                             train_nids, 
#                                             sampler,
#                                             batch_size=1024,
#                                             shuffle=True,
#                                             drop_last=False,
#                                             num_workers=4)

In [ ]:
print(blocks[0].__dict__)

In [ ]:
blocks[0].is_block

In [ ]:
init_unique = np.unique(labels)
for lab in init_unique:
    print(lab)
num_classes = int(len(np.unique(labels)))
onehot_labels = tf.one_hot(indices=labels, depth=num_classes)
unique = np.unique(onehot_labels, axis=0)
for lab in unique:
    print(lab)
    
onehot_dict = {lab: onehot_lab for lab, onehot_lab in zip(init_unique, unique)}
print(onehot_dict)

In [ ]:
print(labels)

In [ ]:
onehot_labels = tf.cast([onehot_dict[l] for l in labels.numpy()], dtype=tf.int64)
print(onehot_labels)

In [ ]:
fc = dgl.function.sum(msg='m', out='h')

In [ ]:


shuffled_labels = tf.random.shuffle(labels, seed=1)
print(shuffled_labels)

shuffled_labels = tf.random.shuffle(labels, seed=1)
print(shuffled_labels)

shuffled_onehot_labels = tf.random.shuffle(onehot_labels, seed=1)
print(shuffled_onehot_labels)

In [10]:
from tensorboard.plugins.hparams import api as hp
import json

In [13]:
layers_config = {'out_feats': [16, 7],
                'activations': ['relu', None]}
print(layers_config)
json_layers_config = json.dumps(layers_config)
print(json_layers_config)
new_layers_config = json.loads(json_layers_config)
print(new_layers_config)



{'out_feats': [16, 7], 'activations': ['relu', None]}
{"out_feats": [16, 7], "activations": ["relu", null]}
{'out_feats': [16, 7], 'activations': ['relu', None]}
